# Klassifizierung eines kompletten Videos

In [9]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

def video_classifier(video_path):
    #Klassenbenennung für das print() am Ende
    class_names = ["von der Seite", "von vorne", "von oben", "von unten"]
    #Liste die mit den frame-predicitons des Videos befüllt wird
    predictions = [0,0,0,0]
    #Frame-Counter
    frames = 0
    #Summe der Softmax Werte
    sum_softmax = 0
    # das ausgewählte Modell wird geladen
    model = tf.keras.models.load_model('/home/dl4/models/model_64_color_few_15')
    #"starte" Timer sobald Modell ausgewählt wurde
    start = time.time()
    # das ausgewählte Video wird geladen
    vidcap = cv2.VideoCapture(video_path)
    
    #Videotrennung in Frames wird initiert
    success, image = vidcap.read()
    while success:
        # einzelnes Frame wird entnommen
        success,image = vidcap.read()
        # sollte es kein images mehr geben, bricht die while-Schleife ab
        if success != True:
            break
        image = cv2.resize(image, (64, 64))
        image = image/255
        # das Bild wird in ein tf-gerechtes Format gebracht
        x = np.asarray(image).astype(np.float32)        
        x = np.expand_dims(x, axis=0)
        # Array mit den Score für das jeweilige Label
        score = model.predict(x)
        # Der Score wird von der Softmax funktion auf einen prozentualen Wert überführt
        probability = tf.nn.softmax(score)
        # Das Label mit der höchsten Wahrscheinlichkeit wird als Vorhersage getroffen (z.B. [0.7, 0.1, 0.1, 0.1] <- Label 0 (Side) wird als Vorhersage getroffen)
        prediction = np.argmax(probability)
        # Softmax-Wert des frames wird zu sum_softmax addiert
        sum_softmax += np.max(probability)
        # Liste wird mit den getroffenen Wahrscheinlichkeiten je Frame gefüllt
        predictions[prediction] += 1
        frames += 1
    
    # Zeitmessung nach Vorhersage des letzen Frames
    end = time.time()
    dauer = end - start
    fps = frames/dauer
    # Nominales Label für das print() wird deklariert
    cam_pos = class_names[np.argmax(predictions)]
    # Wahrscheinlichkeit/Sicherheit der Prognose wird berechnet (anhand der getroffenen Vorhersagen, je Frame)
    ws = predictions[np.argmax(predictions)]/frames*100
    #durschnittlicher Softmax-Wert (in Prozent)<- Aussage über die Konfidenz der Klassifizierung / Wie zuverlässig wurden die Frames im Schnitt klassifiziert
    avg_softmax = (sum_softmax/frames)*100
    # Der Videoname wird aus dem Videopfad entnommen
    word_list = video_path.split("/")
    video = word_list[-1].split(".")
    video_name = video[0]
    
    # Antwortsatz
    print("Das Video\033[1m {} \033[0mist mit einer Wahrscheinlichkeit von\033[1m {}% \033[0m{} gefilmt worden.".format(video_name, int(ws), cam_pos))
    
    # Zusatzinformationen (Dauer des Videos, Bearbeitungsgeschwindigkeit, Verteilung der Label, gesamte Bilderzahl)
    return round(dauer, 2), int(fps), predictions, frames, round(avg_softmax, 2)

In [10]:
a, b, c, d, e = video_classifier("/home/dl4/Praxistest/turm_2.mp4")
print("\nduration:\t\t\t{}\nfps:\t\t\t\t{}\npredictions_distribution:\t{}\nframes:\t\t\t\t{}\navg_softmax:\t\t\t{}".format(a, b, c, d, e))



Das Video turm_2 ist mit einer Wahrscheinlichkeit von 77% von der Seite gefilmt worden.

duration:			4.63
fps:				56
predictions_distribution:	[202, 0, 60, 0]
frames:				262
avg_softmax:			98.33
